# Retrieving Cutouts from the DESI Legacy Imaging Surveys
## Author
Michelle Ding
## Date Created
Thu Jan 20 13:34:14 2022
## Purpose
This script aims to import a list of coordinates (RA and DEC) in degrees decimal form and generate downloadable links. Specific to the object, these links will generate png or fits files of a chosen size.
## Variables
- **HMS_coord:** csv file of the list of objects and their corresponding coordinates in Hour/Degree:Minute:Second form
- **ID:** array of strings containing the names of each object
- **RA_HMS:** array of strings containing the RA coordinate of each object
- **DEC_DMS:** array of strings containing the DEC coordinate of each object
- **RA:** array of floats containing the RA coordinate of each object to 4 dp
- **DEC:** array of floats containing the DEC coordinate of each object to 4 dp
- **filetype:** file type of the output image
- **size:** dimension of the image in pixels (using pixscale = 0.262 pixels/arcsecond)
- **path:** location in which the image will be saved

In [1]:
# import libraries
import numpy as np
import urllib.request as urlopen
import ssl

In [2]:
# import csv file which includes the ID, RA, and DEC of each object
csvfile = input('Enter the path of the csv file') #input the  csv file
HMS_coord = np.genfromtxt(csvfile, delimiter=',', skip_header = 1, dtype='str')
ID = HMS_coord[:,0] # name of objects
RA_HMS = HMS_coord[:,1] # RA in HMS form
DEC_DMS = HMS_coord[:,2] # DEC in DMS form

In [3]:
# convert RA and DEC into degrees decimal form

# Let's convert RA first
RA = np.zeros(len(ID)) # create an empty array

for j in range(len(ID)):
    # np.char.split(RA_HMS)[k] is a list storing Hour, Minute and Second separately
    H, M, S = [float(i) for i in np.char.split(RA_HMS)[j]] # converts string to float
    RA[j] = round(15*(H + (M/60) + (S/3600)), 4) # returns RA in DD form to 4dp


# Next convert DEC
DEC = np.zeros(len(ID)) # create an empty array

for k in range(len(ID)):
    dec_sign = 1 # assuming by default the DEC coordinate is positive
    # np.char.split(DEC_DMS)[k] is a list storing Degrees, Minute and Second separately
    D, M, S = [float(i) for i in np.char.split(DEC_DMS)[k]] # converts string to float
    if str(D)[0] == '-': # confirms the sign of DEC
        # ensures the negative sign is not used in the calculation
        dec_sign, D = -1, abs(D) 
    deg = D + (M/60) + (S/3600) # returns magnitude of DEC
    DEC[k] = '{0}'.format(deg*dec_sign) # returns DEC in DD form

DEC = np.around(DEC, decimals = 4) #correct DEC in DD form to 4dp

In [4]:
# construct links to download fits or jpeg images

# determine the type of image
filetype = input('Enter the type of file required (fits or jpeg): ')

# determine the size of image
size = input("Enter the size of the image in pixels (e.g. 1'=229 px): ")

# select the path to the location of the image
path = input('Enter the path to the location of the output image: ')

for n in range(len(ID)):
    # generates a link to download image
    link = 'https://www.legacysurvey.org/viewer/' + filetype +\
            '-cutout?ra=' + str(RA[n]) + '&dec=' + str(DEC[n]) +\
            '&size=' + size + '&layer=ls-dr9&pixscale=0.262&bands=grz'
    
    def main(): # function to save images as different names depending on the size
        ssl._create_default_https_context = ssl._create_unverified_context
        if size == '229': # 1' images
            r = urlopen.urlretrieve(link, path + ID[n] + "_1'." + filetype)
        if size == '687': # 3' images
            r = urlopen.urlretrieve(link, path + ID[n] + "_3'." + filetype)
        
    if __name__ == '__main__':
        main()

Enter the type of file required (fits or jpeg): jpeg
Enter the size of the image in pixels (e.g. 1'=229 px): 229
Enter the path to the location of the output image: E:/GRAVITATIONAL LENSING/AGEL 1' and 3'/test/
